<a href="https://colab.research.google.com/github/bonly/AI/blob/main/qlib7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
为一个计算机本科学位的量化交易完全小白规划一个为期七天的学习路径，主要使用 Qlib 和 Colab，要求计划详细、系统、可执行，包含每天的里程碑、任务清单和主要技术，全程用中文交流，包括文档及代码注释。

## 里程碑

### Subtask:
了解量化交易基础，成功搭建 Colab 环境并安装 Qlib。


In [1]:
import sys, site
from pathlib import Path

try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    if "google.colab" in sys.modules:
        # The Google colab environment is a little outdated. We have to downgrade the pyyaml to make it compatible with other packages
        ! pip install pyyaml==5.4.1
    # reload
    site.main()

In [106]:
# 安装常用的数据科学库
%pip install pandas numpy matplotlib

# 验证 Qlib 是否成功安装
import qlib
print("Qlib 安装成功！")

# 尝试访问 Qlib 的一些基本属性或模块进行验证
try:
    print("Qlib 版本号:", qlib.__version__)
    from qlib.data import D
    print("qlib.data 模块导入成功！")
except AttributeError:
    print("错误：Qlib 模块属性访问失败，请检查 Qlib 安装。")
except ImportError:
    print("错误：无法导入 qlib.data 模块，请检查 Qlib 安装。")

Qlib 安装成功！
Qlib 版本号: 0.9.7
qlib.data 模块导入成功！


## Qlib 数据管理

### Subtask:
理解 Qlib 的数据结构，并成功下载和管理数据。


In [107]:
scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests

    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py", timeout=10) as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)

In [108]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict

In [109]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data_cn --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    sys.path.append(str(scripts_dir))
    from get_data import GetData

    GetData().qlib_data(target_dir=provider_uri, region=REG_CN)

qlib.init(provider_uri=provider_uri, region=REG_CN)



[997:MainThread](2025-08-18 11:49:58,121) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[997:MainThread](2025-08-18 11:49:58,127) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[997:MainThread](2025-08-18 11:49:58,129) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': PosixPath('/root/.qlib/qlib_data/cn_data')}


In [110]:
market = "sh600048"
benchmark = "SH000300"

In [9]:
# try:
#     instruments = D.instruments(market)
#     print(f"成功获取沪深300成分股列表，共 {len(instruments)} 只股票。")
# except Exception as e:
#     print(f"获取沪深300成分股列表失败: {e}")
#     instruments = [] # 如果获取失败，将 instruments 设为空列表以避免后续错误

成功获取沪深300成分股列表，共 2 只股票。


### 第三天：因子工程与分析

**里程碑:** 理解量化因子概念，学习如何使用 Qlib 进行因子计算和分析。

In [111]:
# 导入必要的库
from qlib.data import D

# 定义一个简单的因子：收盘价
# 'close' 是 Qlib 中预定义的价格字段
close_factor = D.features([market], ['$close'])

print("近五天的收盘价数据：")
display(close_factor.tail())
# print(D.features([benchmark], ['$close']))

近五天的收盘价数据：


$close
instrument datetime             
sh600048   2020-09-21  20.127983
           2020-09-22  19.856779
           2020-09-23  19.668118
           2020-09-24  19.302584
           2020-09-25  18.736593

上面的代码演示了如何获取股票收盘价数据，`D.features` 函数是 Qlib 中用于获取指定股票和指定因子数据的主要接口。

接下来，我们尝试计算一个更常见的因子：日收益率。日收益率可以通过今天的收盘价除以昨天的收盘价再减一来计算。在 Qlib 的 Expression2 中，我们可以使用内置函数来实现。

In [112]:
# 计算日收益率因子： (今天的收盘价 / 昨天的收盘价) - 1
# Ref($close, 1) 表示昨天的收盘价
daily_return_factor = D.features([market], ['$close/Ref($close, 1) - 1'])

print("\n近五天的日收益率数据：")
display(daily_return_factor.tail())


近五天的日收益率数据：


$close/Ref($close, 1) - 1
instrument datetime                             
sh600048   2020-09-21                  -0.016138
           2020-09-22                  -0.013474
           2020-09-23                  -0.009501
           2020-09-24                  -0.018585
           2020-09-25                  -0.029322

Qlib 的 Expression2 支持丰富的运算符和函数，可以用来构建各种复杂的因子。例如，我们可以计算一个简单的移动平均因子：

In [113]:
# 计算5日指数移动平均因子：EMA($close, 5)
# EMA 是 Qlib 中计算指数移动平均的函数
ma5_factor = D.features([market], ['EMA($close, 5)'])

print("\n近五天的5日指数移动平均数据：")
display(ma5_factor.tail())


近五天的5日指数移动平均数据：


EMA($close, 5)
instrument datetime                  
sh600048   2020-09-21       19.989307
           2020-09-22       19.945131
           2020-09-23       19.852795
           2020-09-24       19.669390
           2020-09-25       19.358458

计算出因子后，我们需要评估这些因子的有效性。一个常用的方法是计算因子的 IC (Information Coefficient) 值。IC 值衡量了因子值与未来收益率的相关性。Qlib 提供了一些工具来帮助我们进行因子分析。

### 第四天：量化模型构建

**里程碑:** 了解量化模型的构建流程，学习使用 Qlib 定义和训练模型。

#### 1. 定义模型配置 LGB

Qlib 使用 YAML 格式的配置文件来定义模型和其他工作流组件。我们将定义一个使用 `LGB` 的配置。

In [168]:
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
from qlib.utils import init_instance_by_config, flatten_dict
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
import yaml

market = "csi300"
benchmark = "SH000300"

data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}


# # 将配置写入一个临时文件
# with open("task.yaml", "w") as fs:
#   yaml.safe_dump(task, fs)

# print("模型配置文件已创建：task.yaml")

# with open("task.yaml", "r") as fs:
#     ts = yaml.safe_load(fs)
#     print("模型配置文件：", ts)

模型配置文件已创建：task.yaml


#### 2. Linear模型

In [188]:
# 创建一个简单的数据集配置文件 (YAML 格式)
# 这个配置指定了数据的提供者、日期范围、市场以及使用的特征和标签
# qlib.data.dataset.handler DataHandlerLP
task_str = """
dataset:
    class: DatasetH
    module_path: qlib.data.dataset
    # 数据集划分配置：定义训练集、验证集和测试集的时间范围 (移至顶层)
    kwargs:
      segments:
          train: [2008-01-01, 2014-12-31] # 训练集时间范围
          valid: [2015-01-01, 2016-12-31] # 验证集时间范围
          test: [2017-01-01, 2020-09-30] # 测试集时间范围
      handler:
        class: DataHandlerLP
        module_path: qlib.data.dataset.handler
        kwargs:
          start_time: 2008-01-01
          end_time: 2020-09-30
          instruments: csi300 #使用的股票
          # 特征定义：这里我们使用一些 Qlib 内置的基本特征作为示例
          # 在实际中，您会在这里定义您在第三天计算的各种因子
          features:
              - name: $close/Ref($close, 1) - 1 # 日收益率
                resample_method: H
              - name: EMA($close, 5) # 5日指数移动平均 (这里沿用第三天的 EMA 示例)
                resample_method: H
              - name: Std($close, 5) # 5日收盘价标准差
                resample_method: H
              - name: Ranks(Corr($close, $volume, 5)) # 5日收盘价和成交量的相关系数的横截面排序
                resample_method: H
          # 标签定义：例如，预测未来一天的日收益率
          label:
              - name: Ref($close, -1)/$close - 1 # 明天的日收益率 (这里使用了未来数据，仅为示例，实际中需要谨慎处理未来函数)
                resample_method: H

model:
  class: LinearModel
  module_path: qlib.contrib.model.linear

"""

task1 = yaml.safe_load(task_str)

# 将配置写入一个临时文件
with open("task_str.yaml", "w") as fs:
  yaml.safe_dump(task1, fs)
  print("模型配置文件已创建：task_str.yaml")

with open("task_str.yaml", "r") as fs:
  ts = yaml.safe_load(fs)
  print("模型配置文件：", ts)

模型配置文件已创建：task_str.yaml
模型配置文件： {'dataset': {'class': 'DatasetH', 'kwargs': {'handler': {'class': 'DataHandlerLP', 'kwargs': {'end_time': datetime.date(2020, 9, 30), 'features': [{'name': '$close/Ref($close, 1) - 1', 'resample_method': 'H'}, {'name': 'EMA($close, 5)', 'resample_method': 'H'}, {'name': 'Std($close, 5)', 'resample_method': 'H'}, {'name': 'Ranks(Corr($close, $volume, 5))', 'resample_method': 'H'}], 'instruments': 'csi300', 'label': [{'name': 'Ref($close, -1)/$close - 1', 'resample_method': 'H'}], 'start_time': datetime.date(2008, 1, 1)}, 'module_path': 'qlib.data.dataset.handler'}, 'segments': {'test': [datetime.date(2017, 1, 1), datetime.date(2020, 9, 30)], 'train': [datetime.date(2008, 1, 1), datetime.date(2014, 12, 31)], 'valid': [datetime.date(2015, 1, 1), datetime.date(2016, 12, 31)]}}, 'module_path': 'qlib.data.dataset'}, 'model': {'class': 'LinearModel', 'module_path': 'qlib.contrib.model.linear'}}


**注意:** 上述数据集配置中的特征和标签仅为示例。在实际的量化策略开发中，您需要根据您的研究目标和对市场的理解来定义更丰富和有效的因子，并合理设置标签，避免使用未来函数（即在计算因子或标签时使用了未来的数据）。示例中的标签 `Ref($close, -1)/$close - 1` 预测的是明天的收益率，这在回测中是允许的（因为回测模拟的是历史情况），但在实际交易中则需要确保标签的计算只依赖于当前或之前的数据。

#### 3. 训练模型

有了模型配置和数据集配置，我们就可以使用 Qlib 的工作流来训练模型了。Qlib 的 `R` 对象用于管理实验记录和运行工作流。

In [190]:
# model initialization
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# start exp
with R.start(experiment_name="workflow"):
    # 日志
    R.log_params(**flatten_dict(task))
    # 训练
    model.fit(dataset)
    # 保存
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id
    print("recorder id: ", rid)

    # prediction
    recorder = R.get_recorder()
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

[997:MainThread](2025-08-18 15:37:44,096) INFO - qlib.timer - [log.py:127] - Time cost: 169.846s | Loading data Done
[997:MainThread](2025-08-18 15:37:45,426) INFO - qlib.timer - [log.py:127] - Time cost: 0.423s | DropnaLabel Done
[997:MainThread](2025-08-18 15:37:50,854) INFO - qlib.timer - [log.py:127] - Time cost: 5.425s | CSZScoreNorm Done
[997:MainThread](2025-08-18 15:37:50,870) INFO - qlib.timer - [log.py:127] - Time cost: 6.771s | fit & process data Done
[997:MainThread](2025-08-18 15:37:50,873) INFO - qlib.timer - [log.py:127] - Time cost: 176.623s | Init data Done
[997:MainThread](2025-08-18 15:37:50,880) INFO - qlib.workflow - [exp.py:258] - Experiment 896748042358808599 starts running ...
[997:MainThread](2025-08-18 15:37:50,896) INFO - qlib.workflow - [recorder.py:345] - Recorder 180c38f8d2ab44029e5b0189ca2c19a3 starts running under Experiment 896748042358808599 ...
[997:MainThread](2025-08-18 15:37:50,923) INFO - qlib.workflow - [recorder.py:378] - Fail to log the uncommi

Training until validation scores don't improve for 50 rounds
[20]	train's l2: 0.990585	valid's l2: 0.99431
[40]	train's l2: 0.986931	valid's l2: 0.993693
[60]	train's l2: 0.984352	valid's l2: 0.99349
[80]	train's l2: 0.982319	valid's l2: 0.993382
[100]	train's l2: 0.980442	valid's l2: 0.99331
[120]	train's l2: 0.97871	valid's l2: 0.993247
[140]	train's l2: 0.976987	valid's l2: 0.993334
[160]	train's l2: 0.97536	valid's l2: 0.993338
Early stopping, best iteration is:
[122]	train's l2: 0.978519	valid's l2: 0.993238


[997:MainThread](2025-08-18 15:39:37,523) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 896748042358808599
[997:MainThread](2025-08-18 15:39:37,545) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2017-01-03 SH600000   -0.042865
           SH600008    0.005925
           SH600009    0.030596
           SH600010   -0.013973
           SH600015   -0.141758


上面的代码演示了如何在 Qlib 中定义一个简单的线性模型，配置数据集，并训练模型。训练完成后，模型会被保存在 Qlib 的实验记录中，方便后续的回测和分析。

**第四天任务小结:**

*   我们了解了 Qlib 中模型构建的基本流程：定义模型配置 -> 定义数据集配置 -> 训练模型。
*   创建了使用 `LinearModel` 的模型配置。
*   创建了包含示例特征和标签的数据集配置。
*   使用 `qlib.workflow.R` 训练了模型。

接下来，您可以尝试：

*   修改 `dataset_config.yaml`，添加或修改使用的特征，尝试您在第三天计算的其他因子。
*   尝试不同的日期范围进行训练。
*   如果对线性模型不感兴趣，可以查阅 Qlib 文档，尝试使用其他内置的模型，例如 `LGBModel` (LightGBM) 或 `MLPModel` (多层感知机)。这需要修改 `linear_model_config.yaml` 文件中的 `class` 和 `module_path`。

完成这些探索后，我们就准备进入第五天的学习：回测系统与策略实现。

### 第五天：回测系统与策略实现

**里程碑:** 理解回测的重要性，学习使用 Qlib 回测系统验证策略效果。

#### 1. 理解 Qlib 回测系统

Qlib 的回测系统模拟了真实交易过程，考虑了交易成本、滑点等因素。回测的基本流程通常包括：

*   **加载数据:** 使用与模型训练时一致或相似的数据集。
*   **加载模型:** 加载第四天训练并保存的模型。
*   **生成信号:** 使用加载的模型对回测期内的数据进行预测，生成交易信号（例如，预测的未来收益率）。
*   **构建策略:** 根据交易信号定义交易规则（例如，买入预测收益率最高的股票）。
*   **模拟交易:** 回测系统根据策略和交易规则在历史数据上模拟交易过程。
*   **生成报告:** 回测完成后，系统生成详细的回测报告，包含各种风险和收益指标。

#### 2. 配置回测任务

Qlib 的回测任务也通常通过 YAML 格式的配置文件来定义。这个配置文件会包含数据集、模型、策略、回测时间范围、交易成本等信息。

我们将创建一个简单的回测配置文件作为示例。

In [191]:
# 创建一个简单的回测配置文件 (YAML 格式)
# 这个配置将使用一个预设的（或假设存在的）模型和简单策略
backtest_config = """
strategy:
    class: WeightStrategy # 使用一个简单的权重策略
    module_path: qlib.contrib.strategy.weight_strategy
    kwargs:
        model: <model_path> # 这里需要替换为您训练好的模型路径或记录 ID
        decision_type: # 决策类型
        risk_exposure_type: # 风险暴露类型
        long_short: # 多空方向

backtest:
    class: PortfolioBacktest # 使用组合回测
    module_path: qlib.backtest.high_performance.executor
    kwargs:
        # 回测时间范围
        start_time: 2017-01-01
        end_time: 2020-09-30
        # 交易成本设置
        account: 100000000 # 初始资金
        benchmark_config:
            # 基准指数配置
            benchmark: SH000300
            sign: ~
            flex_benchmark: ~
        exchange_usd: # 交易成本
            # flat_commission: 0.0005 # 固定佣金
            # impact_cost: 0.001 # 冲击成本
            buy_cost: 0.0005 # 买入成本
            sell_cost: 0.0015 # 卖出成本 (通常包含印花税)
            # min_tx_cost: 5 # 最小交易成本
        pos_type: Position # 持仓类型
        # Other parameters...


"""

# 注意：上面的 <model_path> 需要替换为您在 Day 4 训练并保存的模型路径或 Qlib 记录 ID。
# 由于我们未能成功训练模型，这里的配置仅为示例结构。
# 在实际应用中，您需要根据您的模型和策略需求填充详细参数。

# 将配置写入一个临时文件
with open("backtest_config.yaml", "w") as f:
    f.write(backtest_config)

print("回测配置文件已创建：backtest_config.yaml")

回测配置文件已创建：backtest_config.yaml


上面的回测配置文件定义了一个简单的**权重策略 (WeightStrategy)** 和一个**组合回测 (PortfolioBacktest)**。

*   **WeightStrategy:** 这是一种根据模型输出的预测信号来分配股票权重的策略。例如，预测收益率越高的股票，分配的权重越大。`<model_path>` 占位符表示需要指定用于生成预测信号的模型。
*   **PortfolioBacktest:** 这是 Qlib 中常用的回测执行器，用于模拟管理一个股票组合的回测过程。您可以在其中配置回测时间范围、初始资金、交易成本等。

#### 3. 运行回测

有了回测配置文件，我们就可以使用 Qlib 来运行回测任务了。同样，我们可以使用 `qlib.workflow.R` 来管理回测实验记录。

**重要提示:** 由于我们没有一个成功训练的模型来替换 `<model_path>`，直接运行以下代码会失败。以下代码仅为了演示回测的执行流程。在您拥有训练好的模型后，需要修改 `backtest_config.yaml` 文件中的 `<model_path>`。

如果您希望在当前环境下运行一个可以执行的回测示例，我们可以尝试使用 Qlib 提供的默认模型或一个非常简单的规则作为信号源，但这需要对回测配置进行相应的调整。为了遵循计划结构，我们先展示标准的配置方式。

In [192]:
import qlib
from qlib.workflow import R
from qlib.utils import init_instance_by_config
import yaml
# 导入 PortfolioBacktest 类，以便在构建配置时引用其路径
from qlib.backtest.high_performance.executor import PortfolioBacktest
from qlib.contrib.strategy.weight_strategy import WeightStrategy


# 初始化 Qlib (如果之前没有初始化，这里需要根据您的provider_uri进行初始化)
# qlib.init(provider_uri='~/.qlib/qlib_data/cn_data', region=REG_CN) # 请根据您的实际路径修改

# 加载回测配置
with open("backtest_config.yaml", "r") as f:
    backtest_config_dict = yaml.safe_load(f)

# --- 构建回测工作流配置字典 ---
# 创建一个字典，其 class 指向一个工作流执行器，并将 strategy 和 backtest 配置作为 kwargs
# 注意：这里我们构建一个符合 init_instance_by_config 期望的结构
# 实际的 workflow class/module_path 可能需要查阅 Qlib 文档
backtest_workflow_config = {
    'class': 'Workflow', # 假设存在一个 Workflow 执行器类
    'module_path': 'qlib.workflow', # 假设其模块路径
    'kwargs': {
        'strategy': backtest_config_dict['strategy'], # 传递 strategy 配置
        'backtest': backtest_config_dict['backtest'] # 传递 backtest 配置
        # 如果 Workflow 类需要其他参数，可以在这里添加
    }
}
# --- 构建回测工作流配置字典结束 ---


# 开始一个 Qlib 实验记录用于回测
with R.start(experiment_name="linear_model_backtest"):
    # 使用 init_instance_by_config 执行回测工作流
    # 将构建好的 backtest_workflow_config 字典传递给 init_instance_by_config
    backtest_result = init_instance_by_config(backtest_workflow_config)

    # 回测结果会自动记录到 R 中

print("\n回测任务配置已加载。请注意，由于模型路径未替换，直接运行会失败。")
print("在您拥有训练好的模型并更新配置文件后，可以运行此单元格进行回测。")
print("此外，'Workflow' 类和 'qlib.workflow' 模块路径是假设的，实际可能需要查阅 Qlib 文档确认。")

[997:MainThread](2025-08-18 15:50:03,055) WARNING - qlib.workflow - [expm.py:231] - No valid experiment found. Create a new experiment with name linear_model_backtest.
[997:MainThread](2025-08-18 15:50:03,062) INFO - qlib.workflow - [exp.py:258] - Experiment 905061498146545918 starts running ...
[997:MainThread](2025-08-18 15:50:03,077) INFO - qlib.workflow - [recorder.py:345] - Recorder 9640afcf35de4bb7913ea08a19474e43 starts running under Experiment 905061498146545918 ...
[997:MainThread](2025-08-18 15:50:03,088) INFO - qlib.workflow - [recorder.py:378] - Fail to log the uncommitted code of $CWD(/content) when run git diff.
[997:MainThread](2025-08-18 15:50:03,098) INFO - qlib.workflow - [recorder.py:378] - Fail to log the uncommitted code of $CWD(/content) when run git status.
[997:MainThread](2025-08-18 15:50:03,116) INFO - qlib.workflow - [recorder.py:378] - Fail to log the uncommitted code of $CWD(/content) when run git diff --cached.
[997:MainThread](2025-08-18 15:50:03,122) INF

KeyError: 'func'

上面的代码演示了如何加载回测配置文件并尝试使用 `init_instance_by_config` 来执行回测任务。

**第五天任务小结:**

*   我们了解了 Qlib 回测系统的基本概念和流程。
*   学习了如何定义一个回测配置文件，包括策略和回测执行器配置。
*   了解了如何使用 `qlib.workflow.R` 运行回测任务。

接下来，您可以尝试：

*   仔细阅读 `backtest_config.yaml` 文件，理解其中各个参数的含义。查阅 Qlib 文档以获取更多关于 `WeightStrategy` 和 `PortfolioBacktest` 的信息。
*   了解 Qlib 支持的其他策略类型和回测执行器。
*   当您拥有一个成功训练的模型后，更新 `backtest_config.yaml` 中的模型路径，然后运行回测代码。

完成这些探索后，我们就准备进入第六天的学习：策略优化与评估。